In [2]:
from Clasificador import Clasificador 
import numpy as np
import math 
from datos import Datos
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import KFold
import EstrategiaParticionado
from EstrategiaParticionado import ValidacionCruzada



In [6]:
def normalizar_atributo(datos_atributo,esDiscreto,mean,std):
    if (esDiscreto):
        return datos_atributo
    else:
        return list(map(lambda x: (x - mean) / std, datos_atributo))

In [22]:
#Version que no normaliza para realizar el analisis requerido
def normalizar_atributo(datos_atributo,esDiscreto,mean,std):
    return datos_atributo

In [7]:
def comparativa_con_sklearn(datos):
    estrategia = EstrategiaParticionado.ValidacionCruzada(10)
    sklearn_cv = KFold(n_splits=10,shuffle=True)
    for k in [1,3,5,11,21,51]:
            knn_classifier = KNeighborsClassifier(n_neighbors=k,algorithm='ball_tree')
            clasificador = ClasificadorKNN(k)
            scores = cross_val_score(knn_classifier, datos.datos[:,:-1],datos.datos[:,-1], cv = sklearn_cv)
            errores = clasificador.validacion(estrategia,datos,clasificador)
            print ("Tasa de acierto k = ",k," :",1 - np.mean(errores))
            print ("Tasa de acierto k = ",k," version sklearn :", np.mean(scores))
            
class ClasificadorKNN (Clasificador):
    datos = np.array([])
    means = []
    stdevs = []
    k = 0
    def __init__(self,k):
        super(ClasificadorKNN,self).__init__()
        self.k = k
            
    def entrenamiento(self, datosTrain, atributosDiscretos, diccionario):
        datos_traspuestos = datosTrain.transpose()
        self.means = list(map(np.mean,datos_traspuestos))
        self.stdevs = list(map(np.std,datos_traspuestos))
        datos_normalizados = map(normalizar_atributo,
                                 datos_traspuestos,
                                 atributosDiscretos,
                                 self.means,
                                 self.stdevs)
        self.datos = np.array(list(datos_normalizados)).transpose()
  
    def __clasifica_dato(self,dato, atributosDiscretos, diccionario):
        def __dist_euclides(dato_entrenamiento):
            def dist_eje (x,y,esDiscreto):
                return int (x != y) if esDiscreto else pow((x - y),2)
            return math.sqrt(sum(map(dist_eje,
                                     dato_entrenamiento,
                                     dato,
                                     atributosDiscretos)))         
        distancias = map(__dist_euclides,self.datos[:,:-1])
        distancias_min = sorted(zip(distancias,self.datos[:,-1]),key = lambda x: x[0])[:self.k]
        vecinos = list(map(lambda x : x[1], distancias_min))
        dominio_clase = list(diccionario[-1].values())
        frecuencias_clases = list(map(vecinos.count,dominio_clase))
        return dominio_clase[frecuencias_clases.index(max(frecuencias_clases))]
                    
    def clasifica (self, datosTest, atributosDiscretos, diccionario):
        datos_test_normalizados = np.array(list(map(normalizar_atributo,
                                                    datosTest.transpose(),
                                                    atributosDiscretos,
                                                    self.means,
                                                    self.stdevs))).transpose()
        return [ self.__clasifica_dato (dato,atributosDiscretos,diccionario)
                 for dato in datos_test_normalizados]

    


In [8]:
wine = Datos ("../ficheros/ConjuntosDatos/wine_proc.data", True)
digits = Datos("../ficheros/ConjuntosDatos/digits.data",True)
example3 = Datos("../ficheros/ConjuntosDatos/example3.data",True)
example4 = Datos("../ficheros/ConjuntosDatos/example4.data",True)
wdbc = Datos("../ficheros/ConjuntosDatos/wdbc.data",True)
print(len(wine.datos))

            

178


In [25]:
comparativa_con_sklearn(wine)

Tasa de acierto k =  1  : 0.721176470588
Tasa de acierto k =  1  version sklearn : 0.768954248366
Tasa de acierto k =  3  : 0.703294117647
Tasa de acierto k =  3  version sklearn : 0.707189542484
Tasa de acierto k =  5  : 0.691529411765
Tasa de acierto k =  5  version sklearn : 0.674836601307
Tasa de acierto k =  11  : 0.709176470588
Tasa de acierto k =  11  version sklearn : 0.685620915033
Tasa de acierto k =  21  : 0.697647058824
Tasa de acierto k =  21  version sklearn : 0.71339869281
Tasa de acierto k =  51  : 0.715058823529
Tasa de acierto k =  51  version sklearn : 0.709803921569


In [30]:
comparativa_con_sklearn(digits)

Tasa de acierto k =  1  : 0.908510638298
Tasa de acierto k =  1  version sklearn : 0.91170212766
Tasa de acierto k =  3  : 0.88085106383
Tasa de acierto k =  3  version sklearn : 0.889361702128
Tasa de acierto k =  5  : 0.88829787234
Tasa de acierto k =  5  version sklearn : 0.884042553191
Tasa de acierto k =  11  : 0.882978723404
Tasa de acierto k =  11  version sklearn : 0.882978723404
Tasa de acierto k =  21  : 0.865957446809
Tasa de acierto k =  21  version sklearn : 0.862765957447
Tasa de acierto k =  51  : 0.829787234043
Tasa de acierto k =  51  version sklearn : 0.820212765957


In [9]:
comparativa_con_sklearn(example3)

Tasa de acierto k =  1  : 0.9825
Tasa de acierto k =  1  version sklearn : 0.9775
Tasa de acierto k =  3  : 0.965
Tasa de acierto k =  3  version sklearn : 0.9675
Tasa de acierto k =  5  : 0.975
Tasa de acierto k =  5  version sklearn : 0.9725
Tasa de acierto k =  11  : 0.9775
Tasa de acierto k =  11  version sklearn : 0.9725
Tasa de acierto k =  21  : 0.9625
Tasa de acierto k =  21  version sklearn : 0.9625
Tasa de acierto k =  51  : 0.9775
Tasa de acierto k =  51  version sklearn : 0.965


In [28]:
comparativa_con_sklearn(example4)

Tasa de acierto k =  1  : 0.95
Tasa de acierto k =  1  version sklearn : 0.97
Tasa de acierto k =  3  : 0.9
Tasa de acierto k =  3  version sklearn : 0.89
Tasa de acierto k =  5  : 0.88
Tasa de acierto k =  5  version sklearn : 0.88
Tasa de acierto k =  11  : 0.79
Tasa de acierto k =  11  version sklearn : 0.8
Tasa de acierto k =  21  : 0.68
Tasa de acierto k =  21  version sklearn : 0.65
Tasa de acierto k =  51  : 0.39
Tasa de acierto k =  51  version sklearn : 0.44


In [29]:
comparativa_con_sklearn(wdbc)

Tasa de acierto k =  1  : 0.924945054945
Tasa de acierto k =  1  version sklearn : 0.920895989975
Tasa de acierto k =  3  : 0.931840659341
Tasa de acierto k =  3  version sklearn : 0.926221804511
Tasa de acierto k =  5  : 0.931346153846
Tasa de acierto k =  5  version sklearn : 0.935056390977
Tasa de acierto k =  11  : 0.933626373626
Tasa de acierto k =  11  version sklearn : 0.935025062657
Tasa de acierto k =  21  : 0.928021978022
Tasa de acierto k =  21  version sklearn : 0.929636591479
Tasa de acierto k =  51  : 0.913736263736
Tasa de acierto k =  51  version sklearn : 0.917481203008
